In [1]:
import pandas as pd
import numpy as np_
import json
import re
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
def get_domain(url):
    return re.match('https?://(.*?)(/|$).*', url).groups()[0]


In [3]:
def save_to_csv(frequencies, name):
    frequencies.to_frame().reset_index().rename(
        columns={'index': 'domain', 0: 'frequency'}
    ).to_csv(f'../data/{name}_lor_domains.csv', index_label='id')

In [4]:
def read_from_csv(name):
    pd.read_csv(f'../data/{name}_lor_domains.csv')

### Chemie

In [5]:
with open('../data/elixier_chemie_all.json', 'r')  as f:
    chemie_json = json.load(f)

In [6]:
chemie_domain_frequencies = pd.Series([get_domain(u['url']) for u in chemie_json], name='url').value_counts()

In [7]:
chemie_domain_frequencies.head()

www.lehrer-online.de                    504
media.sodis.de                          142
www.seilnacht.com                        86
daten.didaktikchemie.uni-bayreuth.de     51
www.lingonetz.de                         46
Name: url, dtype: int64

### Physik

In [8]:
physik_df = pd.read_csv('../data/elixier_physics_all.csv')

In [9]:
physik_domain_frequencies = physik_df.url.apply(get_domain).value_counts()

In [10]:
physik_domain_frequencies.head()

www.leifiphysik.de            822
www.lehrer-online.de          636
www.supra-lernplattform.de     92
www.bildung-lsa.de             80
media.sodis.de                 76
Name: url, dtype: int64

### Combined

In [11]:
out = dict()
for i in chemie_domain_frequencies.index:
    out[i] = [chemie_domain_frequencies[i], 'chemie']
    
for i in physik_domain_frequencies.index:
    if i in out:
        out[i] = [out[i][0] + physik_domain_frequencies[i], 'physik+chemie']
    else:
        out[i] = [physik_domain_frequencies[i], 'physik']

counts_domain = pd.DataFrame(out).transpose().rename(columns={0: 'count', 1: 'domain'}).sort_values(by='count', ascending=False)
counts = counts_domain.reset_index().rename(columns={'index': 'url_domain'})

In [12]:
counts.head()

,url_domain,count,domain
0,www.lehrer-online.de,1140,physik+chemie
1,www.leifiphysik.de,822,physik
2,media.sodis.de,218,physik+chemie
3,www.planet-schule.de,116,physik+chemie
4,www.supra-lernplattform.de,106,physik+chemie


In [16]:
['https://' + d for d in counts.url_domain]

['https://www.lehrer-online.de',
 'https://www.leifiphysik.de',
 'https://media.sodis.de',
 'https://www.planet-schule.de',
 'https://www.supra-lernplattform.de',
 'https://www.lingonetz.de',
 'https://www.seilnacht.com',
 'https://de.wikipedia.org',
 'https://www.bildung-lsa.de',
 'https://www.zum.de',
 'https://www.abi-physik.de',
 'https://static.bildung-rp.de',
 'https://daten.didaktikchemie.uni-bayreuth.de',
 'https://www.komm-mach-mint.de',
 'https://lernarchiv.bildung.hessen.de',
 'https://www.lncu.de',
 'https://www.chemieunterricht.de',
 'https://www.heise.de',
 'https://www.zauberhafte-physik.net',
 'https://www.wissenschaftsjahr.de',
 'https://digitallearninglab.de',
 'https://mp3.bildung.hessen.de',
 'https://www.dlr.de',
 'https://primas.ph-freiburg.de',
 'https://www.science-on-stage.de',
 'https://www.schule-bw.de',
 'https://www.esa.int',
 'https://educ.ethz.ch',
 'https://www.chemie-master.de',
 'https://www.wissenschaft-schulen.de',
 'https://www.forscher-online.de',


In [13]:
def get_email(domain):
    try:
        response = requests.get('https://' + domain)
        return set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
    except:
        return None

In [ ]:
emails = [get_email(d) for d in tqdm(counts.url_domain)]


 17%|███████▏                                 | 102/584 [03:26<02:40,  3.01it/s]


 18%|███████▌                                 | 108/584 [03:29<02:39,  2.98it/s]

### Scrape Emails

In [63]:
requests.exceptions.TooManyRedirects

requests.exceptions.TooManyRedirects

In [68]:


from bs4 import BeautifulSoup
import requests
import requests.exceptions
from urllib.parse import urlsplit
from collections import deque
import re


def get_emails(domain):
    # a queue of urls to be crawled
    new_urls = deque([f'https://{domain}'])

    # a set of urls that we have already crawled
    processed_urls = set()

    # a set of crawled emails
    emails = set()
    
    request_count = 0

    # process urls one by one until we exhaust the queue
    while len(new_urls):
        # move next url from the queue to the set of processed urls
        url = new_urls.popleft()
        processed_urls.add(url)

        # extract base url to resolve relative links
        parts = urlsplit(url)
        base_url = "{0.scheme}://{0.netloc}".format(parts)
        path = url[:url.rfind('/')+1] if '/' in parts.path else url

        # get url's content
#         print("Processing %s" % url)
        try:
            request_count += 1
            response = requests.get(url)
        except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, requests.exceptions.TooManyRedirects):
            # ignore pages with errors
            continue

        # extract all email addresses and add them into the resulting set
        new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
        emails.update(new_emails)
        if len(new_emails) > 5:
            break
            
        if request_count > 10:
            break

        # create a beutiful soup for the html document
        soup = BeautifulSoup(response.text)

        # find and process all the anchors in the document
        link = None
        for anchor in soup.find_all("a"):
            # extract link url from the anchor
            link = anchor.attrs["href"] if "href" in anchor.attrs else ''
        if link is None:
            continue
        # resolve relative links
        if link.startswith('/'):
            link = base_url + link
        elif not link.startswith('http'):
            link = path + link
        # add the new url to the queue if it was not enqueued nor processed yet
        if not link in new_urls and not link in processed_urls:
            new_urls.append(link)

#     print(f'Found {emails}')
    return emails


In [69]:
emails_out = dict()
for d in tqdm(counts.url_domain):
    print(d)
    emails_out[d] = get_emails(d)

  0%|                                                   | 0/584 [00:00<?, ?it/s]

www.lehrer-online.de


  0%|                                           | 1/584 [00:00<08:25,  1.15it/s]

www.leifiphysik.de


  1%|▏                                          | 3/584 [00:02<05:41,  1.70it/s]

...

www.chemikus.de
www.medienwerkstatt-online.de


 17%|███████                                   | 99/584 [10:51<18:45,  2.32s/it]

www.bimsev.de
physikunterricht-online.de


 17%|███████                                   | 99/584 [10:52<53:14,  6.59s/it]


InvalidURL: Failed to parse: https://physikunterricht-online.dejavascript:void(0);